<a href="https://colab.research.google.com/github/AlessioChen/Computer-Vision-Class/blob/main/resnet18_cvlab2_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchinfo import summary
from torch.optim.lr_scheduler import ExponentialLR, MultiStepLR

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


# Utils

In [20]:
import time
import numpy as np
import matplotlib.pyplot as plt
import json
!pip install wandb -qU
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [21]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [22]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [23]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()  # training mode

    losses, accs = [], []
    correct = 0.

    for batch_idx, (data, target) in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        data, target = data.to(device), target.to(device)

        ## zero the parameter gradients
        optimizer.zero_grad()

        ## forward + backward + optimize
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        ## Prediction
        pred = output.argmax(dim=1, keepdim=True)
        correct = torch.eq(pred, target.view_as(pred)).float()
        acc = torch.mean(correct)

        losses.append(loss.detach().cpu().numpy())
        accs.append(acc.detach().cpu().numpy())

    loss_k = np.array(losses).mean()
    acc_k = np.array(accs).mean()

    return loss_k, acc_k

In [24]:
def test(model, device, criterion, test_loader):
    model.eval()  # evaluation mode

    losses, accs = [], []
    correct = 0.

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            ## Compute loss
            output = model(data)
            loss = criterion(output, target)

            ## Prediction
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct = torch.eq(pred, target.view_as(pred)).float()
            # correct += pred.eq(target.view_as(pred)).sum().item()
            acc = torch.mean(correct)

            ## Update loss and accuracy
            losses.append(loss.detach().cpu().numpy())
            accs.append(acc.detach().cpu().numpy())

    loss_k = np.array(losses).mean()
    acc_k = np.array(accs).mean()

    return loss_k, acc_k

In [25]:

def train_loop(train_loader, test_loader, model, criterion, device,
               lr, momentum, max_epochs, do_test=True):
    """" Training loop with SGD """

    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    losses_train, accs_train = [], []
    losses_test, accs_test = [], []

    _start = time.time()
    _epoch_time = time.time()

    for epoch in range(1, max_epochs + 1):
        loss_train, acc_train = train(model, device, train_loader, criterion, optimizer)
        print(f"Epoch: {epoch}, Learning rate: {get_lr(optimizer):.6f}")
        print(f"Training - Loss: {loss_train:.4f}, Accuracy: {acc_train:.2f}, Runtime: {(time.time() - _epoch_time):.2f}")
        losses_train.append(loss_train)
        accs_train.append(acc_train)

        if do_test:
            loss_test, acc_test = test(model, device, criterion, test_loader)
            losses_test.append(loss_test)
            accs_test.append(acc_test)
            print(f"Test - Loss: {loss_test:.4f}, Accuracy: {acc_test:.2f}")

        _epoch_time = time.time()

    _end = time.time()
    print(f"Done! - Runtime: {(_end-_start):.2f} seconds")

    # test_class(model, device, criterion, testloader)

    if do_test:
        return losses_train, accs_train, losses_test, accs_test
    else:
        return losses_train, accs_train

In [26]:
def train_loop_sched(train_loader, test_loader, model, criterion, device,
                     optimizer, scheduler=None, max_epochs=10, do_test=True):
    """ Training loop with custom optimizer and optional learning rate scheduler """

    losses_train, accs_train = [], []
    losses_test, accs_test = [], []

    _start = time.time()
    _epoch_time = time.time()

    # init of wandb
    wandb.init(
      project='cv-lab2',
      config= {
          "learning_rate": get_lr(optimizer),
          "dataset": "CIFAR10",
          "architecture": "ResNet18",
          "epochs": max_epochs,
      },
      entity='alessiochen98-university-of-florence',
      name='reset18'
    )

    # epoch number only for logging
    for epoch in range(1, max_epochs + 1):
        ## Training
        loss_train, acc_train = train(model, device, train_loader, criterion, optimizer)
        print(f"Epoch: {epoch}, Learning rate: {get_lr(optimizer):.6f}")
        print(f"Training - Loss: {loss_train:.4f}, Accuracy: {acc_train:.3f}, Runtime: {(time.time() - _epoch_time):.2f}")

        wandb.log({
            "train_loss": loss_train,
            "train_accuracy": acc_train,
        })

        losses_train.append(loss_train)
        accs_train.append(acc_train)
        # Learning rate scheduler
        if scheduler is not None:
            scheduler.step()
        ## Test/Validation
        if do_test:
            loss_test, acc_test = test(model, device, criterion, test_loader)
            losses_test.append(loss_test)
            accs_test.append(acc_test)
            print(f"Test - Loss: {loss_test:.4f}, Accuracy: {acc_test:.3f}")

            wandb.log({
              "test_loss": loss_train,
              "test_accuracy": acc_train,
            })

        _epoch_time = time.time()

    _end = time.time()
    print(f"Done! - Runtime: {(_end-_start):.2f} seconds")

    if do_test:
        return losses_train, accs_train, losses_test, accs_test
    else:
        return losses_train, accs_train


In [27]:
def test_class(model, device, criterion, test_loader, classes):
    model.eval()  # configura il modello in evaluation mode

    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # prepare to count overall predictions
    losses, accs = [], []
    correct = 0.

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            ## Fit data and compute loss
            output = model(data)
            loss = criterion(output, target)

            ## Prediction
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

            # compute overall accuracy
            correct = torch.eq(pred, target.view_as(pred)).float()
            acc = torch.mean(correct)
            # update loss and accuracy
            losses.append(loss.detach().cpu().numpy())
            accs.append(acc.detach().cpu().numpy())

            for target_i, pred_i in zip(target, pred):
                if target_i == pred_i:
                    correct_pred[classes[target_i]] += 1
                total_pred[classes[target_i]] += 1

    loss_final = np.array(losses).mean()
    acc_final = np.array(accs).mean()
    print(f"Final loss: {loss_final:.4f}, Accuracy: {acc_final:.3f}")
    print("-------")

    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [28]:
def simple_diagnostic(max_epochs, losses_train, accs_train):
    epochs_seq = np.arange(1, max_epochs + 1)
    # epoch_labels =

    # plot only training loss and accuracy
    fig, ax = plt.subplots()
    # fig.suptitle("Training performance")
    fig.suptitle("Training loss and accuracy againts epochs")

    color = "tab:blue"
    ax.plot(epochs_seq, losses_train, label="loss", color=color)
    # ax.set_title("Training loss and accuracy againts epochs")
    ax.grid("both")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss", color=color)
    ax.tick_params(axis="y", labelcolor=color)
    ax.set_xticks(np.arange(1, max_epochs+1, step=2))
    ax.set_xticklabels(np.arange(1, max_epochs + 1, 2))

    color = "tab:red"
    ax_1 = ax.twinx()
    ax_1.plot(epochs_seq, accs_train, label="accuracy", color="tab:red")
    ax_1.set_ylabel("Accuracy", color="tab:red")
    ax_1.tick_params(axis="y", labelcolor=color)

In [29]:
def diagnostic(losses_train, accs_train, losses_test, accs_test):
    ## left side training performance
    ## right side test performance
    max_epochs = len(losses_train)
    epochs_seq = np.arange(1, max_epochs + 1)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4), layout="constrained")
    fig.suptitle("CNN performance over MNIST")

    ## 1) train loss (first y axis) and accuracy (second y axis)
    color = "tab:blue"
    axs[0].plot(epochs_seq, losses_train, label="loss", color=color)
    axs[0].set_title("Training loss and accuracy againts epochs")
    axs[0].grid("both")
    axs[0].set_xlabel("Epochs")
    axs[0].set_ylabel("Loss", color=color)
    axs[0].tick_params(axis="y", labelcolor=color)

    color = "tab:red"
    axs0_1 = axs[0].twinx()
    axs0_1.plot(epochs_seq, accs_train, label="accuracy", color="tab:red")
    axs0_1.set_ylabel("Accuracy", color="tab:red")
    axs0_1.tick_params(axis="y", labelcolor=color)

    ## 2) test loss (first y axis) and accuracy (second y axis)
    color = "tab:blue"
    axs[1].plot(epochs_seq, losses_test, label="loss", color=color)
    axs[1].set_title("Test loss and accuracy againts epochs")
    axs[1].grid("both")
    axs[1].set_xlabel("Epochs")
    axs[1].set_ylabel("Loss", color=color)
    axs[1].tick_params(axis="y", labelcolor=color)

    color = "tab:red"
    axs1_1 = axs[1].twinx()
    axs1_1.plot(epochs_seq, accs_test, label="accuracy", color="tab:red")
    axs1_1.set_ylabel("Accuracy", color="tab:red")
    axs1_1.tick_params(axis="y", labelcolor=color)

In [30]:
def multiple_diagnostic_single(loss_acc_dict, max_epochs=10):
    # loss_acc_dict = {"Solver1": [loss, acc]...}
    epochs_seq = np.arange(1, max_epochs + 1)
    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']

    fig, ax = plt.subplots()
    ax_1 = ax.twinx()
    # fig.suptitle("CNN training performance over CIFAR10")
    fig.suptitle("Training loss and accuracy againts epochs")

    for i, (solver_name, perf) in enumerate(loss_acc_dict.items()):

        color = colors[i]
        ## plot loss function performance
        ax.plot(epochs_seq, perf[0], label=solver_name, color=color)
        ax.grid("both")
        ax.set_xlabel("Epochs")
        ax.set_ylabel("Loss")
        ax.tick_params(axis="y")
        ax.set_xticks(np.arange(1, max_epochs+1, step=2))
        ax.set_xticklabels(np.arange(1, max_epochs + 1, 2))

        ## plot accuracy performance
        ax_1.plot(epochs_seq, perf[1], label=solver_name, color=color)
        ax_1.set_ylabel("Accuracy")
        ax_1.tick_params(axis="y")

    ax.legend()


In [31]:
def multiple_diagnostic(loss_acc_dict, max_epochs=None, title_left="Training loss against epochs",
                        title_right="Test accuracy against epochs", fig_title=""):
    # loss_acc_dict = {"Solver1": [loss, acc]...}

    if max_epochs is None:
        max_epochs = len(next(iter(loss_acc_dict.values()))[0])
        # print(max_epochs)

    epochs_seq = np.arange(1, max_epochs + 1)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4), layout="constrained")
    fig.suptitle(fig_title)

    for solver_name, perf in loss_acc_dict.items():

        # plot loss function performance
        axs[0].plot(epochs_seq, perf[0], label=solver_name)
        axs[0].grid("both")
        axs[0].set_title(title_left)
        # axs[0].set_xlabel("Epochs")
        # axs[0].set_ylabel("Loss")
        axs[0].tick_params(axis="y")
        axs[0].set_xticks(np.arange(1, max_epochs+1, step=2))
        axs[0].set_xticklabels(np.arange(1, max_epochs+1, 2))

        # plot accuracy performance
        axs[1].plot(epochs_seq, perf[1], label=solver_name)
        axs[1].grid("both")
        axs[1].set_title(title_right)
        # axs[1].set_xlabel("Epochs")
        # axs[1].set_ylabel("Accuracy")
        axs[1].tick_params(axis="y")
        axs[1].set_xticks(np.arange(1, max_epochs+1, step=2))
        axs[1].set_xticklabels(np.arange(1, max_epochs+1, 2))

    axs[0].legend()
    axs[1].legend()


In [32]:
def save_to_json(loss_acc_dict, file_name):
    """ Save diagnostic to JSON file for reuse """
    # loss_acc_dict: {"Solver1": [[list of np.float32], [list of np.float32]]}
    # file_name: "file_name.json"

    # convert to float for serialization
    dict_converted = {k: [[float(x) for x in seq] for seq in v] for k, v in loss_acc_dict.items()}

    with open(file_name, "w") as file:
        json.dump(dict_converted, file)

# Resnet 18

In [15]:
head_loss_acc = {}

In [16]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# batch_size = 64
batch_size = 128
max_epochs = 20

opt_dict = dict(lr=0.01, momentum=0.9, nesterov=True, weight_decay=5e-4)

criterion = nn.CrossEntropyLoss()

In [17]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # some augmentation
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    # transforms.Resize(224)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    # transforms.Resize(224)
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

# create a split for train/validation. We can use early stop
trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2,
                                          drop_last=True, pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2,
                                        drop_last=False, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2,
                                         drop_last=False, pin_memory=True)

100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [18]:
resnet18 = models.resnet18(weights="DEFAULT")

resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)
resnet18 = resnet18.to(device)

# print(summary(resnet18))
print(f"Trainable parameters: {count_trainable_parameters(resnet18)}")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 79.8MB/s]


Trainable parameters: 11181642


In [19]:
optimizer = optim.SGD(resnet18.parameters(), **opt_dict)
# scheduler = ExponentialLR(optimizer, 0.7)
scheduler = MultiStepLR(optimizer, [5, 10, 15], 0.1)

stats = train_loop_sched(trainloader, testloader, resnet18, criterion, device, optimizer, scheduler, max_epochs)

head_loss_acc["Full"] = [stats[0], stats[3]]

print("=========")
test_class(resnet18, device, criterion, testloader, classes)

Epoch: 1, Learning rate: 0.010000
Training - Loss: 1.0765, Accuracy: 0.628, Runtime: 29.60
Test - Loss: 0.7949, Accuracy: 0.728
Epoch: 2, Learning rate: 0.010000
Training - Loss: 0.7372, Accuracy: 0.748, Runtime: 19.28
Test - Loss: 0.6654, Accuracy: 0.775
Epoch: 3, Learning rate: 0.010000
Training - Loss: 0.6357, Accuracy: 0.781, Runtime: 20.67
Test - Loss: 0.6191, Accuracy: 0.789
Epoch: 4, Learning rate: 0.010000
Training - Loss: 0.5793, Accuracy: 0.801, Runtime: 24.88
Test - Loss: 0.5813, Accuracy: 0.802
Epoch: 5, Learning rate: 0.010000
Training - Loss: 0.5336, Accuracy: 0.815, Runtime: 19.24
Test - Loss: 0.5768, Accuracy: 0.808
Epoch: 6, Learning rate: 0.001000
Training - Loss: 0.4311, Accuracy: 0.851, Runtime: 20.56
Test - Loss: 0.4804, Accuracy: 0.838
Epoch: 7, Learning rate: 0.001000
Training - Loss: 0.3964, Accuracy: 0.863, Runtime: 19.09
Test - Loss: 0.4684, Accuracy: 0.843
Epoch: 8, Learning rate: 0.001000
Training - Loss: 0.3780, Accuracy: 0.869, Runtime: 20.41
Test - Loss: 